<a href="https://colab.research.google.com/github/mohameddhameem/NLPWithPyTorch/blob/master/Learn_BERT_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Install the additional libaries.
! pip install -U git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-w760l7jy
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-w760l7jy
     |████████████████████████████████| 3.7MB 9.8MB/s 
     |████████████████████████████████| 1.0MB 32.0MB/s 
     |████████████████████████████████| 870kB 37.7MB/s 
  Created wheel for transformers: filename=transformers-2.5.1-cp36-none-any.whl size=534520 sha256=07f75b602b09fef4332680e80de7bf6824a4991a48d09d6ff645863815465042
  Stored in directory: /tmp/pip-ephem-wheel-cache-5qqu4739/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=eadcf6a5ff338674c8455faae391798c7ee4a298941968b881900dacb59fa970
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built transformers sacremoses


In [3]:
from itertools import chain
from collections import namedtuple

import numpy as np
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
# A tokenizer will split the text into the appropriate sub-parts (aka. tokens).
# Depending on how the pre-trained model is trained, the tokenizers defers.
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Example of a tokenized input after WordPiece Tokenization.
text = "[CLS] my dog is cute [SEP] he likes playing [SEP]"
print(tokenizer.wordpiece_tokenizer.tokenize(text))


['[CLS]', 'my', 'dog', 'is', 'cute', '[SEP]', 'he', 'likes', 'playing', '[SEP]']


In [5]:
"playing" in tokenizer.wordpiece_tokenizer.vocab

True

In [6]:
print("slacking" in tokenizer.wordpiece_tokenizer.vocab)

False


In [7]:
text = "[CLS] my dog is cute [SEP] he likes slacking [SEP]"
tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text) # There, we see the ##ing token!
print(tokenized_text)

['[CLS]', 'my', 'dog', 'is', 'cute', '[SEP]', 'he', 'likes', 'slack', '##ing', '[SEP]']


In [8]:
token_indices = tokenizer.convert_tokens_to_ids(tokenized_text)
token_indices

[101, 2026, 3899, 2003, 10140, 102, 2002, 7777, 19840, 2075, 102]

In [9]:
import numpy as np

# We need to create an array that indicates the end of sentences, delimited by [SEP]
text = "[CLS] my dog is cute [SEP] he likes slacking [SEP]"
tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text)  # There, we see the ##ing token!

# First we find the indices of `[SEP]`, and incrementally adds it up. 
# Here's some Numpy gymnastics... 
# Thanks to @divakar https://stackoverflow.com/a/58316889/610569
m = np.asarray(tokenized_text) == "[SEP]"
segments_ids = m.cumsum()-m

tokens_tensor, segments_tensors = torch.tensor([token_indices]), torch.tensor([segments_ids])

# See the type change?
print(tokens_tensor.shape, type(token_indices), type(tokens_tensor))
print(segments_tensors.shape, type(segments_ids), type(segments_tensors))

torch.Size([1, 11]) <class 'list'> <class 'torch.Tensor'>
torch.Size([1, 11]) <class 'numpy.ndarray'> <class 'torch.Tensor'>


In [10]:
tokens_tensor, segments_tensors = torch.tensor([token_indices]), torch.tensor([segments_ids])

# See the type change?
print(tokens_tensor.shape, type(token_indices), type(tokens_tensor))
print(segments_tensors.shape, type(segments_ids), type(segments_tensors))

torch.Size([1, 11]) <class 'list'> <class 'torch.Tensor'>
torch.Size([1, 11]) <class 'numpy.ndarray'> <class 'torch.Tensor'>


# Lets convert our input text to an array of number

## First, we load the pre-trained model

In [11]:
import torch 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# When using the BERT model for "encoding", i.e. convert string to array of floats, 
# we use the `BertModel` object from pytorch transformer library.
model = BertModel.from_pretrained('bert-base-uncased')
model.eval(); model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor.to(device), segments_tensors.to(device))
    
print(encoded_layers)

tensor([[[ 0.0737,  0.1229, -0.1612,  ...,  0.1235,  0.2381,  0.0880],
         [ 0.5442,  0.6884, -0.0744,  ..., -0.0346,  0.1388, -0.4474],
         [ 0.7117,  0.6298, -0.0084,  ...,  0.1541,  0.1260, -0.1969],
         ...,
         [ 0.6583,  0.4991, -0.1022,  ...,  0.2715, -0.2863, -0.4839],
         [-0.1440, -0.3067, -0.3331,  ...,  0.4313,  0.1080,  0.0214],
         [ 0.9325,  0.2506,  0.0805,  ...,  0.1434, -0.8708, -0.4659]]],
       device='cuda:0')


In [13]:
encoded_layers.shape

torch.Size([1, 11, 768])

# The BERT model is very good at fill-in-the-blank task

The BERT model is trained using a "cloze" task where words are randomly replaced with the [MASK] symbols and the model learns to adjust its parameters such that it learns which words are most probable to fit into the [MASK] symbols.

When using the BERT model for "guessing missing words", we use the BertForMaskedLM object from pytorch transformer library. Here's an example if we blank out words in the sentence, BERT is able to find the appropriate word to fill it in.

In [14]:
# Load the model.
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval(); model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [15]:
# We need to create an array that indicates the end of sentences, delimited by [SEP]
text = "[CLS] please don't let the [MASK] out of the [MASK] . [SEP]"
tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text)
token_indices = tokenizer.convert_tokens_to_ids(tokenized_text)

# Create the segment indices.
m = np.asarray(tokenized_text) == "[SEP]"
segments_ids = m.cumsum()-m

# Convert them to the arrays to pytorch tensors.
tokens_tensor, segments_tensors = torch.tensor([token_indices]), torch.tensor([segments_ids])

# Apply the model to the inputs.
with torch.no_grad(): # You can take this context manager to mean that we're not training.
    outputs, *_ = model(tokens_tensor.to(device), 
                        token_type_ids=segments_tensors.to(device))

outputs.shape

torch.Size([1, 14, 30522])

In [0]:
# Apply the model to the inputs.
with torch.no_grad(): # You can take this context manager to mean that we're not training.
    outputs, *_ = model(tokens_tensor.to(device), token_type_ids=segments_tensors.to(device))

In [17]:
outputs.shape

torch.Size([1, 14, 30522])

In [18]:
print(tokenized_text)

['[CLS]', 'please', 'don', "##'", '##t', 'let', 'the', '[MASK]', 'out', 'of', 'the', '[MASK]', '.', '[SEP]']


In [19]:
# Lets remember our original masked sentence.
print(tokenized_text)
# We have to check where the masked token is from the original text. 
mask_index = tokenized_text.index('[MASK]') 
assert mask_index == 7 # The 7th token.

# Then we fetch the vector for the 7th value, 
# The [0, mask_index] refers to accessing vector of vocab_size for
# the 0th sentence, mask_index-th token.
output_value = outputs[0, mask_index]

# As a sanity check we can see that the shape of the output_value
# is the same as the `vocab_size` from the outputs' shape.
assert int(output_value.shape[0]) == len(tokenizer.wordpiece_tokenizer.vocab)

['[CLS]', 'please', 'don', "##'", '##t', 'let', 'the', '[MASK]', 'out', 'of', 'the', '[MASK]', '.', '[SEP]']


In [20]:
# Lets recap the original sentence with the masked word.
print(text)

# We have to check where the first masked token is from the original text. 
mask_index = tokenized_text.index('[MASK]') 
output_value = outputs[0, mask_index]

## We use torch.argmax to get the index with the highest value.
mask_word_in_vocab = int(torch.argmax(output_value))
print(tokenizer.convert_ids_to_tokens([mask_word_in_vocab]))

[CLS] please don't let the [MASK] out of the [MASK] . [SEP]
['cat']


In [21]:
# Lets recap the original sentence with the masked word.
print(text)

# We have to check where the masked tokens are from the original text. 
for mask_index, token in enumerate(tokenized_text):
    if token == '[MASK]':
        output_value = outputs[0, mask_index]
        mask_word_in_vocab = int(torch.argmax(output_value))
        print(tokenizer.convert_ids_to_tokens([mask_word_in_vocab]))

[CLS] please don't let the [MASK] out of the [MASK] . [SEP]
['cat']
['way']


# Lets make the fill-in-the-blank feature into a function.

In [0]:
def fill_in_the_blanks(text, model, tokenizer, return_str=False):
    tokenized_text = tokenizer.wordpiece_tokenizer.tokenize(text)
    token_indices = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Create the segment indices.
    m = np.asarray(tokenized_text) == "[SEP]"
    segments_ids = m.cumsum()-m
    # Convert them to the arrays to pytorch tensors.
    tokens_tensor = torch.tensor([token_indices]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    
    # Apply the model to the inputs.
    with torch.no_grad(): # You can take this context manager to mean that we're not training.
        outputs, *_ = model(tokens_tensor, token_type_ids=segments_tensors)
    
    output_tokens = []
    for mask_index, token_id in enumerate(token_indices):
        token = tokenizer.convert_ids_to_tokens([token_id])[0]
        if token == '[MASK]':
            output_value = outputs[0, mask_index]
            # The masked word index in the vocab.
            mask_word_in_vocab = int(torch.argmax(output_value))
            token = tokenizer.convert_ids_to_tokens([mask_word_in_vocab])[0]
        output_tokens.append(token)
        
    return " ".join(output_tokens).replace(" ##", "").replace(" ' t ", "'t ") if return_str else output_tokens

In [23]:
# Load the model.
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval(); model.to(device)

text = "[CLS] please don't let the [MASK] out of the [MASK] . [SEP]"
print(fill_in_the_blanks(text, model, tokenizer, return_str=True))

[CLS] please don't let the cat out of the way . [SEP]


In [24]:
text = "[CLS] i like to drink beer and eat [MASK] . [SEP]"
print(fill_in_the_blanks(text, model,tokenizer, return_str=True))

[CLS] i like to drink beer and eat meat . [SEP]


In [25]:
text = "[CLS] i like to drink coffee and eat [MASK] . [SEP]"
print(fill_in_the_blanks(text, model, tokenizer, return_str=True))

[CLS] i like to drink coffee and eat it . [SEP]


# Fine-tuning BERT models

In [0]:
phoenix_turtle = """Truth may seem but cannot be;\nBeauty brag but ’tis not she;\nTruth and beauty buried be."""
sonnet20 = """A woman’s face with Nature’s own hand painted\nHast thou, the master-mistress of my passion;\nA woman’s gentle heart, but not acquainte\nWith shifting change, as is false women’s fashion;"""
sonnet1 = """From fairest creatures we desire increase,\nThat thereby beauty’s rose might never die,\nBut as the riper should by time decease,\nHis tender heir might bear his memory:"""
sonnet73 = """In me thou see’st the glowing of such fire,\nThat on the ashes of his youth doth lie,\nAs the death-bed whereon it must expire,\nConsum’d with that which it was nourish’d by."""
venus_adonis = """It shall be cause of war and dire events,\nAnd set dissension ‘twixt the son and sire;\nSubject and servile to all discontents,\nAs dry combustious matter is to fire:\nSith in his prime Death doth my love destroy,\nThey that love best their loves shall not enjoy\n"""
sonnet29 = """When, in disgrace with fortune and men’s eyes,\nI all alone beweep my outcast state,\nAnd trouble deaf heaven with my bootless cries,\nAnd look upon myself and curse my fate,"""
sonnet130 = """I have seen roses damask’d, red and white,\nBut no such roses see I in her cheeks;\nAnd in some perfumes is there more delight\nThan in the breath that from my mistress reeks."""
sonnet116 = """Love’s not Time’s fool, though rosy lips and cheeks\nWithin his bending sickle’s compass come;\nLove alters not with his brief hours and weeks,\nBut bears it out even to the edge of doom."""
sonnet18 = """But thy eternal summer shall not fade\nNor lose possession of that fair thou ow’st;\nNor shall Death brag thou wander’st in his shade,\nWhen in eternal lines to time thou grow’st;\nSo long as men can breathe or eyes can see,\nSo long lives this, and this gives life to thee."""
anthony_cleo = """She made great Caesar lay his sword to bed;\nHe plowed her, and she cropped."""

shakespeare = [phoenix_turtle, sonnet20, sonnet1, sonnet73, venus_adonis,
              sonnet29, sonnet130, sonnet116, sonnet18, anthony_cleo]

In [27]:
from transformers import BertConfig, BertForMaskedLM, BertTokenizer

# Load the BERT model.
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
model.to(device)
# Load the BERT Tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# Load the BERT Config.
config = BertConfig.from_pretrained('bert-large-uncased')

In [28]:
truth = "that on the ashes of his youth doth lie"
masked_text = "[CLS] that on the ashes of his youth [MASK] lie"
print(fill_in_the_blanks(masked_text, model, tokenizer, return_str=True))

[CLS] that on the ashes of his youth , lie


## Prepare Data set for custom text

In [0]:
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
import torch.nn.functional as F

class TextDataset(Dataset):
    def __init__(self, texts, tokenizer):
        """
        :param texts: A list of documents, each document is a list of strings.
        :rtype texts: list(string)
        """
        tokenization_process = lambda s: tokenizer.build_inputs_with_special_tokens(
                                             tokenizer.convert_tokens_to_ids(
                                                 tokenizer.tokenize(s.lower())))
        pad_sent = lambda x: np.pad(x, (0,tokenizer.max_len_single_sentence - len(x)), 'constant', 
                                    constant_values=tokenizer.convert_tokens_to_ids(tokenizer.pad_token))
        self.examples = torch.tensor([pad_sent(tokenization_process(doc)) for doc in texts])

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

In [0]:
# Initialize the Dataset object.
train_dataset = TextDataset(shakespeare, tokenizer)
# Initalize the DataLoader object, `batch_size=2` means reads 2 poems at a time.
dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=2)

In [32]:
# 10 poems with 510 tokens per poems, 
# if poem has <510, pad with the 0th index.
train_dataset.examples.shape

# For each batch, we read 2 poems at a time.
print(next(iter(dataloader)).shape)

torch.Size([2, 510])


In [33]:
# An example of a batch.
next(iter(dataloader))

tensor([[ 101, 1037, 2450,  ...,    0,    0,    0],
        [ 101, 2009, 4618,  ...,    0,    0,    0]])

## Mask Tokens in Random - 10% of the time it'll random replace the masked word with another word, and 10% of the time it'll keep the original

https://github.com/huggingface/transformers/blob/master/examples/run_lm_finetuning.py#L143

In [0]:
def mask_tokens(inputs, tokenizer, mlm_probability=0.8):
    """ Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original. """

    if tokenizer.mask_token is None:
        raise ValueError(
            "This tokenizer does not have a mask token which is necessary for masked language modeling. Remove the --mlm flag if you want to use this tokenizer."
        )

    labels = inputs.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, mlm_probability)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    if tokenizer._pad_token is not None:
        padding_mask = labels.eq(tokenizer.pad_token_id)
        probability_matrix.masked_fill_(padding_mask, value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked tokens

    # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
    inputs[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    # 10% of the time, we replace masked input tokens with random word
    indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
    random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
    inputs[indices_random] = random_words[indices_random]

    # The rest of the time (10% of the time) we keep the masked input tokens unchanged
    return inputs, labels

In [0]:
from transformers.optimization import AdamW
from transformers.optimization import get_linear_schedule_with_warmup as WarmupLinearSchedule

Arguments = namedtuple('Arguments', ['learning_rate', 'weight_decay', 'adam_epsilon', 'num_warmup_steps', 
                                     'max_steps', 'num_train_epochs'])

args = Arguments(learning_rate=5e-3, weight_decay=0.0, adam_epsilon=1e-8, num_warmup_steps=0, # Optimizer arguments
                 max_steps=20, num_train_epochs=50  # Training routine arugments
                )  

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = WarmupLinearSchedule(optimizer, num_warmup_steps=args.num_warmup_steps, num_training_steps=args.max_steps)

In [36]:
for _e in range(args.num_train_epochs):
    print('Epoch:', _e, '\t' ,end='')
    for step, batch in enumerate(iter(dataloader)):
        print(step, end=', ')

        optimizer.zero_grad()
        # Randomly mask the tokens 80% of the time. 
        inputs, labels = mask_tokens(batch, tokenizer)
        inputs, labels = inputs.to(device), labels.to(device)
        # Initialize the model to train mode.
        model.train()
        # Feed forward the inputs through the models.
        loss, _ = model(inputs, masked_lm_labels=labels)

        # Backpropagate the loss.
        loss.backward()
        # Step through the optimizer.
        optimizer.step()
    print()

Epoch: 0 	0, 1, 2, 3, 4, 
Epoch: 1 	0, 1, 2, 3, 4, 
Epoch: 2 	0, 1, 2, 3, 4, 
Epoch: 3 	0, 1, 2, 3, 4, 
Epoch: 4 	0, 1, 2, 3, 4, 
Epoch: 5 	0, 1, 2, 3, 4, 
Epoch: 6 	0, 1, 2, 3, 4, 
Epoch: 7 	0, 1, 2, 3, 4, 
Epoch: 8 	0, 1, 2, 3, 4, 
Epoch: 9 	0, 1, 2, 3, 4, 
Epoch: 10 	0, 1, 2, 3, 4, 
Epoch: 11 	0, 1, 2, 3, 4, 
Epoch: 12 	0, 1, 2, 3, 4, 
Epoch: 13 	0, 1, 2, 3, 4, 
Epoch: 14 	0, 1, 2, 3, 4, 
Epoch: 15 	0, 1, 2, 3, 4, 
Epoch: 16 	0, 1, 2, 3, 4, 
Epoch: 17 	0, 1, 2, 3, 4, 
Epoch: 18 	0, 1, 2, 3, 4, 
Epoch: 19 	0, 1, 2, 3, 4, 
Epoch: 20 	0, 1, 2, 3, 4, 
Epoch: 21 	0, 1, 2, 3, 4, 
Epoch: 22 	0, 1, 2, 3, 4, 
Epoch: 23 	0, 1, 2, 3, 4, 
Epoch: 24 	0, 1, 2, 3, 4, 
Epoch: 25 	0, 1, 2, 3, 4, 
Epoch: 26 	0, 1, 2, 3, 4, 
Epoch: 27 	0, 1, 2, 3, 4, 
Epoch: 28 	0, 1, 2, 3, 4, 
Epoch: 29 	0, 1, 2, 3, 4, 
Epoch: 30 	0, 1, 2, 3, 4, 
Epoch: 31 	0, 1, 2, 3, 4, 
Epoch: 32 	0, 1, 2, 3, 4, 
Epoch: 33 	0, 1, 2, 3, 4, 
Epoch: 34 	0, 1, 2, 3, 4, 
Epoch: 35 	0, 1, 2, 3, 4, 
Epoch: 36 	0, 1, 2, 3, 4, 
Epoch: 37 	

In [37]:
truth = "That on the ashes of his youth doth lie"
masked_text = "[CLS] That on the ashes of his youth [MASK] lie [SEP]"
print(fill_in_the_blanks(masked_text, model, tokenizer, return_str=True))

[CLS] [UNK] on the ashes of his youth , lie [SEP]


In [38]:
labels

tensor([[-100, 2293, 1521,  ..., -100, -100, -100],
        [-100, 2021, -100,  ..., -100, -100, -100]], device='cuda:0')

In [39]:
labels.to(device)

tensor([[-100, 2293, 1521,  ..., -100, -100, -100],
        [-100, 2021, -100,  ..., -100, -100, -100]], device='cuda:0')

In [40]:
inputs[1][0]

tensor(101, device='cuda:0')